In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#check GPU (optional)
!nvidia-smi

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as tt
import torch.optim as optim
from torchsummary import summary

ROOT_PATH = '/content/drive/MyDrive/QMUL/NN/code/nn_group5'
sys.path.append(ROOT_PATH)
from my_module import functions as myf
from my_module import make_dataset as mds
model_path = '/content/drive/MyDrive/QMUL/NN/code/model_para/'

myf.torch_fix_seed()

In [ ]:
#get dataloader
train_dl = mds.get_dl(
    data='training', 
    bs=64, 
    shuffle=True, 
    transform=tt.Compose([tt.ToTensor(), tt.Normalize(0.5, 0.5)])
)
validation_dl = mds.get_dl(
    data='validation', 
    bs=64, 
    shuffle=False, 
    transform=tt.Compose([tt.ToTensor(), tt.Normalize(0.5, 0.5)])
)
test_dl = mds.get_dl(
    data='test', 
    bs=64, 
    shuffle=False, 
    transform=tt.Compose([tt.ToTensor(), tt.Normalize(0.5, 0.5)])
)
dev = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#check some samples
dict_classes={
    0: 'Angry',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happy',
    4: 'Sad',
    5: 'Surprised',
    6: 'Neutral'
  }
fig, axes = plt.subplots(1, 5, tight_layout=True)
for i in range(5):
    img, label = train_dl.dataset[np.random.randint(0,10000)]
    axes[i].imshow(img.squeeze(), cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(dict_classes[label])


In [ ]:
#NN model
class Base_CNN(nn.Module):
    def __init__(self):
        super(Base_CNN, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 9 * 9, 120)  # 9*9 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Base_CNN_dropout(nn.Module):
    def __init__(self):
        super(Base_CNN_dropout, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 9 * 9, 120)  # 9*9 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
model = Base_CNN().to(dev)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
epochs = 30

Base_CNN_train_acc, Base_CNN_val_acc = myf.fit(
    model,
    optimizer,
    epochs,
    train_dl,
    validation_dl,
    print_loss=True
)
torch.save(model.state_dict(), model_path + 'Base_CNN.pth')
np.save('/content/drive/MyDrive/QMUL/NN/code/result/base_CNN_train', Base_CNN_train_acc)
np.save('/content/drive/MyDrive/QMUL/NN/code/result/base_CNN_val', Base_CNN_val_acc)

In [ ]:
test_acc = myf.test_loop(
    test_dl,
    model,
    nn.CrossEntropyLoss())
print(test_acc)

In [ ]:
myf.plot_cfmat(
    model, 
    validation_dl, 
    dict_classes, 
    savefig=True, 
    name='1_BaseCNN_cfmat')

In [ ]:
model = Base_CNN_dropout().to(dev)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
epochs = 30

Base_CNN_dropout_train_acc, Base_CNN_dropout_val_acc = myf.fit(
    model,
    optimizer,
    epochs,
    train_dl,
    validation_dl,
    print_loss=True
)

torch.save(model.state_dict(), model_path + 'Base_CNN_dropout.pth')
np.save('/content/drive/MyDrive/QMUL/NN/code/result/base_CNN_dropout_train', Base_CNN_dropout_train_acc)
np.save('/content/drive/MyDrive/QMUL/NN/code/result/base_CNN_dropout_val', Base_CNN_dropout_val_acc)

In [ ]:
test_acc = myf.test_loop(
    test_dl,
    model,
    nn.CrossEntropyLoss())
print(test_acc)

In [ ]:
myf.plot_cfmat(
    model, 
    validation_dl, 
    dict_classes, 
    savefig=True, 
    name='2_BaseCNN_do_cfmat')